# November 16th Meeting
## CG rewritten on CPU




# Redesign structure

```jl
struct variables
    Nx::Int64
    Ny::Int64
    N::Int64
    hx::Float64
    hy::Float64
    x
    y
    alpha1::Float64
    alpha2::Float64
    alpha3::Float64
    alpha4::Float64
    beta::Float64
end

Nx = 1001
Ny = 1001
N = Nx * Ny

variable = variables(Nx,Ny,N,(1/(Nx-1)),(1/(Ny-1)),0:1/(Nx-1):1,0:1/(Ny-1):1,-1.0,-1.0,-13/(1/(Nx-1)),-13/(1/(Ny-1)),1.0)



# intermediate results
mutable struct intermediates
    Nx::Int64
    Ny::Int64
    N::Int64
    Vol2Face::Array{Array{Float64,1},1}
    du_x::Array{Float64,1}
    du_y::Array{Float64,1}
    du_ops::Array{Float64,1}
    du1::Array{Float64,1}
    du2::Array{Float64,1}
    du3::Array{Float64,1}
    du4::Array{Float64,1}
    du5::Array{Float64,1}
    du6::Array{Float64,1}
    du7::Array{Float64,1}
    du8::Array{Float64,1}
    du9::Array{Float64,1}
    du10::Array{Float64,1}
    du11::Array{Float64,1}
    du12::Array{Float64,1}
    du13::Array{Float64,1}
    du14::Array{Float64,1}
    du15::Array{Float64,1}
    du16::Array{Float64,1}
    du17::Array{Float64,1}
    du0::Array{Float64,1}
    du::Array{Float64,1}
end

intermediate = intermediates(Nx,Ny,N,[zeros(N),zeros(N),zeros(N),zeros(N)],zeros(N),zeros(N),zeros(N),zeros(N),zeros(N),zeros(N),zeros(N),zeros(N),zeros(N),zeros(N),zeros(N),zeros(N),zeros(N),zeros(N),zeros(N),zeros(N),zeros(N),zeros(N),zeros(N),zeros(N),zeros(N),zeros(N))


```

## CG on CPU
```jl
function myMAT_beta!(du::AbstractVector, u::AbstractVector,variable,intermediate)
    D2x_beta!(u,variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,intermediate.du_x)
    D2y_beta!(u,variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,intermediate.du_y)

    intermediate.du_ops .= intermediate.du_x .+ intermediate.du_y
    BySy_beta!(u,variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,intermediate.du1)
    VOLtoFACE_beta!(intermediate.du1,1,variable.Nx,variable.Ny,variable.N,intermediate.Vol2Face)
    Hyinv_beta!(intermediate.Vol2Face[1],variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,variable.alpha1,intermediate.du3)     #compute action of P1  .= for faster assignment

    VOLtoFACE_beta!(intermediate.du1,2,variable.Nx,variable.Ny,variable.N,intermediate.Vol2Face)
    Hyinv_beta!(intermediate.Vol2Face[2],variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy, variable.alpha2,intermediate.du6)
    
    #compute action of P2
    VOLtoFACE_beta!(u,3,variable.Nx,variable.Ny,variable.N,intermediate.Vol2Face)
    BxSx_tran_beta!(intermediate.Vol2Face[3],variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,intermediate.du8)
    Hxinv_beta!(intermediate.du8,variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy, variable.beta, intermediate.du9)
    Hxinv_beta!(intermediate.Vol2Face[3],variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,variable.alpha3,intermediate.du11)   #compute action of P3

    VOLtoFACE_beta!(u,4,variable.Nx,variable.Ny,variable.N,intermediate.Vol2Face)
    BxSx_tran_beta!(intermediate.Vol2Face[4],variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,intermediate.du13)
    Hxinv_beta!(intermediate.du13,variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,variable.beta,intermediate.du14)
    Hxinv_beta!(intermediate.Vol2Face[4],variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,variable.alpha4,intermediate.du16)  #compute action of P4
    intermediate.du0 .= intermediate.du_ops .+ intermediate.du3 .+ intermediate.du6 .+ intermediate.du9 .+ intermediate.du11 .+ intermediate.du14 .+ intermediate.du16 #Collect together
    Hy_beta!(intermediate.du0,variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,intermediate.du17)
	Hx_beta!(intermediate.du17,variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy, -1.0,intermediate.du)
    return intermediate.du
end

function myMAT_beta_no_return!(du::AbstractVector, u::AbstractVector,variable,intermediate)
    D2x_beta!(u,variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,intermediate.du_x)
    D2y_beta!(u,variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,intermediate.du_y)

    intermediate.du_ops .= intermediate.du_x .+ intermediate.du_y
    BySy_beta!(u,variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,intermediate.du1)
    VOLtoFACE_beta!(intermediate.du1,1,variable.Nx,variable.Ny,variable.N,intermediate.Vol2Face)
    Hyinv_beta!(intermediate.Vol2Face[1],variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,variable.alpha1,intermediate.du3)     #compute action of P1  .= for faster assignment

    VOLtoFACE_beta!(intermediate.du1,2,variable.Nx,variable.Ny,variable.N,intermediate.Vol2Face)
    Hyinv_beta!(intermediate.Vol2Face[2],variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy, variable.alpha2,intermediate.du6)
    
    #compute action of P2
    VOLtoFACE_beta!(u,3,variable.Nx,variable.Ny,variable.N,intermediate.Vol2Face)
    BxSx_tran_beta!(intermediate.Vol2Face[3],variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,intermediate.du8)
    Hxinv_beta!(intermediate.du8,variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy, variable.beta, intermediate.du9)
    Hxinv_beta!(intermediate.Vol2Face[3],variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,variable.alpha3,intermediate.du11)   #compute action of P3

    VOLtoFACE_beta!(u,4,variable.Nx,variable.Ny,variable.N,intermediate.Vol2Face)
    BxSx_tran_beta!(intermediate.Vol2Face[4],variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,intermediate.du13)
    Hxinv_beta!(intermediate.du13,variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,variable.beta,intermediate.du14)
    Hxinv_beta!(intermediate.Vol2Face[4],variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,variable.alpha4,intermediate.du16)  #compute action of P4
    intermediate.du0 .= intermediate.du_ops .+ intermediate.du3 .+ intermediate.du6 .+ intermediate.du9 .+ intermediate.du11 .+ intermediate.du14 .+ intermediate.du16 #Collect together
    Hy_beta!(intermediate.du0,variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy,intermediate.du17)
	Hx_beta!(intermediate.du17,variable.Nx,variable.Ny,variable.N,variable.hx,variable.hy, -1.0,intermediate.du)
end
```

## Performance

### Old CG function
```jl
julia> @benchmark conjugate_beta(myMAT_beta!,r,b,container,var,intermediate,100)
BenchmarkTools.Trial: 
  memory estimate:  806.03 MiB
  allocs estimate:  4549
  --------------
  minimum time:     2.356 s (0.08% GC)
  median time:      2.442 s (0.08% GC)
  mean time:        2.433 s (0.09% GC)
  maximum time:     2.501 s (0.09% GC)
  --------------
  samples:          3
  evals/sample:     1
  ```


  ### New CG function
  ```jl
  julia> @benchmark conjugate_beta_no_return(myMAT_beta_no_return!,r,b,variable,intermediate,100)
BenchmarkTools.Trial: 
  memory estimate:  33.75 MiB
  allocs estimate:  818
  --------------
  minimum time:     1.694 s (0.00% GC)
  median time:      1.699 s (0.00% GC)
  mean time:        1.707 s (0.00% GC)
  maximum time:     1.728 s (0.00% GC)
  --------------
  samples:          3
  evals/sample:     1
  ```


### How to further speedup?
1. Parallelize independent steps within CG
2. Parallelize matrix-free operators in CG